### Imports:


In [5]:
import numpy as np
import cv2
from skimage.feature import hog
import matplotlib.pyplot as plt
import joblib
import matplotlib.image as mpimg
import pandas as pd
import glob
from sklearn.preprocessing import StandardScaler
from skimage.io import imread
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
%matplotlib inline

In [6]:
### HOG feature extraction:

In [7]:
def extract_features(imgs, cspace='RGB', size = (64,64)):
    features = []
    for filename in imgs:
        image = imread(filename)
        if size != (64,64):
            image = cv2.resize(image, size)
        features.append(
            np.ravel(
                cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9) \
                    .compute(get_feature_space(image, cspace))
            )
        )
    return features

In [8]:
def get_feature_space(img, cspace):
    if cspace != 'RGB':
        if cspace == 'HLS':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif cspace == 'YCrCb':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        elif cspace == 'HSV':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif cspace == 'YUV':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif cspace == 'Lab':
            features = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
        return features
def get_hog_features(img, cspace):
    return np.ravel(
        cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9) \
            .compute(get_feature_space(img, cspace))
    )

In [ ]:

vehicles = glob.glob('Data/vehicles/*/*.png')
non_vehicles = glob.glob('Data/non-vehicles/*/*.png')
vehicle_features = extract_features(vehicles, cspace='YUV')
non_vehicle_features = extract_features(non_vehicles, cspace='YUV')

def plot_features(vehicle_features, non_vehicle_features):
    vehicle_features[0].shape
    if len(vehicle_features) > 0:
        X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64) 
        X_scaler = StandardScaler().fit(X)
        scaled_X = X_scaler.transform(X)
        vehicle_ind = np.random.randint(0, len(vehicles))
        fig = plt.figure(figsize=(12,4))
        plt.subplot(131)
        plt.imshow(mpimg.imread(vehicles[vehicle_ind]))
        plt.title('original image')
        plt.subplot(132)
        plt.plot(scaled_X[vehicle_ind])
        plt.title('scaled features')
        fig.tight_layout()
    return scaled_X, X_scaler
        
scaled_X, X_scaler = plot_features(vehicle_features, non_vehicle_features)  

In [ ]:
### splitting dataset

In [ ]:
#horizontal stack ones for cars and otherwise zero
labels = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))
#20% test images 
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, labels, test_size=0.2, random_state=4000)

In [3]:
##training model with MLP


In [4]:
#Multi-layer Perceptron classifier model

mlp = MLPClassifier(random_state=999)
mlp.fit(X_train, y_train)
print('MLP results')
print('accuracy on training data: ', mlp.score(X_train, y_train))
print('accuracy on test data: ', mlp.score(X_test, y_test))
prediction = mlp.predict(X_test[0].reshape(1, -1))

NameError: name 'MLPClassifier' is not defined

In [ ]:
# saving trained data 
joblib.dump(mlp, 'mlp1.pkl')
joblib.dump(X_scaler, 'scaler1.pkl')


In [ ]:
# loading trained data file 
mlp = joblib.load('mlp1.pkl')
X_scaler = joblib.load('scaler1.pkl')